<a href="https://colab.research.google.com/github/jonlwowski012/StockSellRulesChecker/blob/master/StockScreener/Screener.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
! git clone https://github.com/jonlwowski012/StockSellRulesChecker.git
! pip install -r StockSellRulesChecker/StockScreener/requirements.txt
! pip install -U git+https://github.com/mariostoev/finviz
! pip install "tqdm==4.43.0"

fatal: destination path 'StockSellRulesChecker' already exists and is not an empty directory.
  Cloning https://github.com/mariostoev/finviz to /tmp/pip-req-build-zqmowg2b
  Running command git clone -q https://github.com/mariostoev/finviz /tmp/pip-req-build-zqmowg2b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for finviz: filename=finviz-1.3.4-cp37-none-any.whl size=16979 sha256=1058809cc9b90299a93b3e19ab0d83b1d0a3e73f9ff81e192f55886863af3561
  Stored in directory: /tmp/pip-ephem-wheel-cache-8j9ux7fi/wheels/98/cc/6a/68893eb662f0f196f8e153e9ce4424a8ae9e1f022fd64e4d6d
Successfully built finviz
  Found existing installation: finviz 1.3.4
    Uninstalling finviz-1.3.4:
      Successfully uninstalled finviz-1.3.4


     |████████████████████████████████| 61kB 2.9MB/s 
  Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1


In [1]:
from finviz.screener import Screener
import nest_asyncio
import finviz
import pandas_datareader as pdr
import yfinance as yf
yf.pdr_override() # <== that's all it takes :-)

import datetime
import stockquotes
import pandas as pd
import tqdm
nest_asyncio.apply()

import time
from tqdm.contrib.concurrent import process_map


In [15]:
eps_5year_over5pct = False
eps_qoq_over5pct = False
eps_yoy_over5pct = False
sales_5years_over5pct = False
sales_qoq_over5pct = False

filters = []
if eps_5year_over5pct:
  filters.append('fa_eps5years_o5')
if eps_qoq_over5pct:
  filters.append('fa_epsqoq_o5')
if eps_yoy_over5pct:
  filters.append('fa_epsyoy_o5')
if sales_5years_over5pct:
  filters.append('fa_sales5years_o5')
if sales_qoq_over5pct:
  filters.append('fa_salesqoq_o5')

stock_list = Screener(filters=filters, table='Performance', order='price')  

In [16]:
print(len(stock_list.data))

7971


In [17]:

def moving_average(yahoo_df, days, delta=0):
    df = yahoo_df.reset_index()
    curr_date = df['Date'].max() - datetime.timedelta(days=delta)
    end_date = curr_date - datetime.timedelta(days=days) - datetime.timedelta(days=delta)
    
    df_days = df[df['Date'] >= end_date]
    df_days = df_days[df_days['Date'] <= curr_date]
    return float(df_days['Close'].mean())

def week52_low_high(yahoo_df):
    df = yahoo_df.reset_index()
    curr_date = df['Date'].max()
    end_date = curr_date - datetime.timedelta(days=365)
    
    df_days = df[df['Date'] >= end_date]
    return float(df_days['Close'].max()), float(df_days['Close'].min())

def SMA200_slope_positive_rule(yahoo_df, days=21):
  for day in range(days):
    if moving_average(yahoo_df, days=200, delta=day) > moving_average(yahoo_df, days=200, delta=day+1):
      continue
    else:
      return False
  return True

def screen_stock(stock):
  if stock["Ticker"] == "":
    return
  screened_stocks[stock['Ticker']] = {}
  finviz_stats = finviz.get_stock(stock['Ticker'])
  try:
      yahoo_df = pdr.get_data_yahoo(stock['Ticker'], interval = "d")
  except:
    return

  SMA200_value = moving_average(yahoo_df, days=200)
  SMA150_value = moving_average(yahoo_df, days=150)
  SMA50_value = moving_average(yahoo_df, days=50)
  SMA200_percent = float(finviz_stats['SMA200'].replace("%",""))
  SMA50_percent = float(finviz_stats['SMA50'].replace("%",""))
  volume = float(finviz_stats['Volume'].replace(",",""))

  try:
    EPS_ttm = float(finviz_stats['EPS (ttm)'])
  except:
    EPS_ttm = 0

  try:
    EPS_QoQ_percent = float(finviz_stats['EPS Q/Q'].replace("%",""))
  except:
    EPS_QoQ_percent = 0

  try:
    Sales_QoQ_percent = float(finviz_stats['Sales Q/Q'].replace("%",""))
  except:
    Sales_QoQ_percent = 0

  try:
    inst_own = float(finviz_stats['Inst Own'].replace("%",""))
  except:
    inst_own = 0
  
  shares_outstanding = finviz_stats['Shs Outstand']
  if 'M' in shares_outstanding:
    shares_outstanding = float(shares_outstanding.replace("M", ""))*1e6
  elif 'B' in shares_outstanding:
    shares_outstanding = float(shares_outstanding.replace("B", ""))*1e9
  elif 'T' in shares_outstanding:
    shares_outstanding = float(shares_outstanding.replace("T", ""))*1e12
  else:
    shares_outstanding = 0

  prev_close = float(finviz_stats['Prev Close'].replace("$",""))
  week52_high, week52_low = week52_low_high(yahoo_df)
  
  screened_stocks[stock['Ticker']]['SMA200_value'] = SMA200_value
  screened_stocks[stock['Ticker']]['SMA150_value'] = SMA150_value
  screened_stocks[stock['Ticker']]['SMA50_value'] = SMA50_value
  screened_stocks[stock['Ticker']]['SMA200_percent'] = SMA200_percent
  screened_stocks[stock['Ticker']]['SMA50_percent'] = SMA50_percent
  screened_stocks[stock['Ticker']]['EPS_QoQ_percent'] = EPS_QoQ_percent
  screened_stocks[stock['Ticker']]['EPS_ttm'] = EPS_ttm
  screened_stocks[stock['Ticker']]['Sales_QoQ_percent'] = Sales_QoQ_percent
  screened_stocks[stock['Ticker']]['prev_close'] = prev_close
  screened_stocks[stock['Ticker']]['week52_high'] = week52_high
  screened_stocks[stock['Ticker']]['week52_low'] = week52_low
  screened_stocks[stock['Ticker']]['Inst. Ownership'] = inst_own
  screened_stocks[stock['Ticker']]['Shares Outstanding'] = shares_outstanding
  screened_stocks[stock['Ticker']]['volume'] = volume

  # volume*price > $15mm rule
  if volume*prev_close > 15000000:
    vol_price_rule = True
  else:
    vol_price_rule = False
  screened_stocks[stock['Ticker']]['vol_price_rule'] = vol_price_rule

  # EPS TTM > $1 Rule
  if EPS_ttm >= 1.0:
    eps_ttm_rule = True
  else:
    eps_ttm_rule = False
  screened_stocks[stock['Ticker']]['eps_ttm_rule'] = eps_ttm_rule

  # EPS QoQ Yearly > 18% rule
  if EPS_QoQ_percent >= 18:
    eps_QoQ_yearly_rule = True
  else:
    eps_QoQ_yearly_rule = False
  screened_stocks[stock['Ticker']]['eps_QoQ_yearly_rule'] = eps_QoQ_yearly_rule

  # Sales QoQ Yearly > 25% rule
  if Sales_QoQ_percent >= 25:
    sales_QoQ_yearly_rule = True
  else:
    sales_QoQ_yearly_rule = False
  screened_stocks[stock['Ticker']]['sales_QoQ_yearly_rule'] = sales_QoQ_yearly_rule

  # Shares Outstanding <= 25 mil
  if shares_outstanding <= 25e6:
    shares_outstanding_rule = True
  else:
    shares_outstanding_rule = False
  screened_stocks[stock['Ticker']]['shares_outstanding_rule'] = shares_outstanding_rule

  # Institutional Ownership < 35%
  if 0.05 <= inst_own <= 35:
    inst_ownership_rule = True
  else:
    inst_ownership_rule = False
  screened_stocks[stock['Ticker']]['inst_ownership_rule'] = inst_ownership_rule

  # Positive 200d MA positive
  SMA200_slope_rule = SMA200_slope_positive_rule(yahoo_df, days=21)
  screened_stocks[stock['Ticker']]['SMA200_slope_rule'] = SMA200_slope_rule
      
  # 150d MA greater than 200d MA
  if SMA150_value > SMA200_value:
      SMA150_greater_SMA200_rule = True
  else:
      SMA150_greater_SMA200_rule = False
  screened_stocks[stock['Ticker']]['SMA150_greater_SMA200_rule'] = SMA150_greater_SMA200_rule
      
  # 50d MA greater than 150d MA
  if SMA50_value > SMA150_value:
      SMA50_greater_SMA150_rule = True
  else:
      SMA50_greater_SMA150_rule = False
  screened_stocks[stock['Ticker']]['SMA50_greater_SMA150_rule'] = SMA50_greater_SMA150_rule
      
  # Close above 50d MA
  if prev_close > SMA50_value:
      close_greater_SMA50_rule = True
  else:
      close_greater_SMA50_rule = False
  screened_stocks[stock['Ticker']]['close_greater_SMA50_rule'] = close_greater_SMA50_rule
      
  # 52 week high low span rule
  if 0.75*week52_high > 1.25*week52_low:
      week52_span_rule = True
  else:
      week52_span_rule = False
  screened_stocks[stock['Ticker']]['week52_span_rule'] = week52_span_rule
  
  # Close above 52 week high - 25%
  if prev_close > 0.75*week52_high:
      close_above_52weekhigh_rule = True
  else:
      close_above_52weekhigh_rule = False
  screened_stocks[stock['Ticker']]['close_above_52weekhigh_rule'] = close_above_52weekhigh_rule
  return screened_stocks

screened_stocks = {}
results = process_map(screen_stock, stock_list, max_workers=16)



    


TypeError: ignored

In [25]:
screened_stocks = {}
for d in results:
  if d is not None:
    screened_stocks.update(d)
  
output_list = []
for stock in screened_stocks.keys():
    cols = ["Ticker"] + list(screened_stocks[stock].keys())
    temp_list = []
    temp_list.append(stock)
    for rule in screened_stocks[stock].keys():
        temp_list.append(screened_stocks[stock][rule])
    output_list.append(temp_list)
        
df_out = pd.DataFrame(output_list,columns=cols)

In [26]:
rules = [col for col in df_out.columns if "rule" in col]
df_out['Passed Tests'] = (df_out[list(rules)]).sum(1)

In [27]:
df_out.sort_values('Passed Tests', ascending=False)

,Ticker,SMA200_value,SMA150_value,SMA50_value,SMA200_percent,SMA50_percent,EPS_QoQ_percent,EPS_ttm,Sales_QoQ_percent,prev_close,week52_high,week52_low,Inst. Ownership,Shares Outstanding,volume,vol_price_rule,eps_ttm_rule,eps_QoQ_yearly_rule,sales_QoQ_yearly_rule,shares_outstanding_rule,inst_ownership_rule,SMA200_slope_rule,SMA150_greater_SMA200_rule,SMA50_greater_SMA150_rule,close_greater_SMA50_rule,week52_span_rule,close_above_52weekhigh_rule,Passed Tests
942,TRMT,3.955561,4.320731,5.052722,78.55,28.40,67.0,1.08,27.8,5.92,6.080000,1.67,25.6,8210000.0,27772.0,False,True,True,True,True,True,True,True,True,True,True,True,11
2124,FFHL,7.567626,8.740673,11.136667,71.17,5.94,291.6,2.39,5.9,11.80,14.760000,1.58,9.2,3270000.0,18880.0,False,True,True,False,True,True,True,True,True,True,True,True,10
548,RCON,1.600072,1.827404,2.675556,152.54,57.04,50.2,-0.63,137.6,3.62,4.100000,0.79,2.6,4800000.0,312591.0,False,False,True,True,True,True,True,True,True,True,True,True,10
2063,ZKIN,3.025187,3.607587,6.167222,353.52,115.91,-123.1,-0.05,65.0,8.77,11.200000,0.69,2.8,16560000.0,86506502.0,True,False,False,True,True,True,True,True,True,True,True,True,10
953,BNSO,4.987266,5.248654,6.073056,43.35,1.97,240.2,0.07,64.2,6.11,7.690000,1.94,3.3,4650000.0,5420.0,False,False,True,True,True,True,True,True,True,True,True,True,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2155,BTWN,14.037183,14.037183,13.985278,-16.56,-17.86,0.0,0.00,0.0,11.75,17.370001,10.70,0.0,59500000.0,875538.0,False,False,False,False,False,False,False,False,False,False,False,False,0
1116,TLMD,8.225229,7.905632,7.779444,-20.40,-5.30,0.0,0.00,-11.3,7.24,10.985000,6.81,76.1,31950000.0,290024.0,False,False,False,False,False,False,False,False,False,False,False,False,0
2171,FUSN,12.263381,12.233558,11.718056,-7.55,0.16,-189.0,0.00,0.0,11.31,17.780001,11.02,80.3,41680000.0,255120.0,False,False,False,False,False,False,False,False,False,False,False,False,0
1812,QELL,12.219250,12.219250,12.091667,-16.62,-18.17,0.0,0.00,0.0,10.21,14.350000,10.21,0.0,37950000.0,431206.0,False,False,False,False,False,False,False,False,False,False,False,False,0


In [29]:
df_out[df_out["Passed Tests"] >= 10]

,Ticker,SMA200_value,SMA150_value,SMA50_value,SMA200_percent,SMA50_percent,EPS_QoQ_percent,EPS_ttm,Sales_QoQ_percent,prev_close,week52_high,week52_low,Inst. Ownership,Shares Outstanding,volume,vol_price_rule,eps_ttm_rule,eps_QoQ_yearly_rule,sales_QoQ_yearly_rule,shares_outstanding_rule,inst_ownership_rule,SMA200_slope_rule,SMA150_greater_SMA200_rule,SMA50_greater_SMA150_rule,close_greater_SMA50_rule,week52_span_rule,close_above_52weekhigh_rule,Passed Tests
548,RCON,1.600072,1.827404,2.675556,152.54,57.04,50.2,-0.63,137.6,3.62,4.10,0.79,2.60,4.800000e+06,312591.0,False,False,True,True,True,True,True,True,True,True,True,True,10
798,GGB,4.337410,4.513077,4.660000,31.36,9.87,951.3,0.25,42.9,5.07,5.42,1.72,29.90,1.700000e+09,14850142.0,True,False,True,True,False,True,True,True,True,True,True,True,10
936,ORC,5.367482,5.452404,5.660278,15.81,8.20,403.1,0.04,100.0,5.91,6.04,2.33,28.10,9.432000e+07,5799512.0,True,False,True,True,False,True,True,True,True,True,True,True,10
942,TRMT,3.955561,4.320731,5.052722,78.55,28.40,67.0,1.08,27.8,5.92,6.08,1.67,25.60,8.210000e+06,27772.0,False,True,True,True,True,True,True,True,True,True,True,True,11
953,BNSO,4.987266,5.248654,6.073056,43.35,1.97,240.2,0.07,64.2,6.11,7.69,1.94,3.30,4.650000e+06,5420.0,False,False,True,True,True,True,True,True,True,True,True,True,10
996,SID,4.838058,5.443750,6.232222,59.09,1.64,257.0,0.49,50.1,6.57,7.15,1.07,2.60,1.380000e+09,4995158.0,True,False,True,True,False,True,True,True,True,True,True,True,10
1140,TURN,6.200073,6.339231,7.101667,25.49,8.23,243.4,0.11,536.1,7.51,7.56,3.33,22.80,1.037000e+07,43460.0,False,False,True,True,True,True,True,True,True,True,True,True,10
1365,ONCT,4.216619,4.984135,6.937778,140.53,40.58,67.8,-0.95,128.6,8.36,9.00,1.51,21.60,4.928000e+07,1993255.0,True,False,True,True,False,True,True,True,True,True,True,True,10
1759,SLGG,2.864748,3.207115,4.784444,272.27,141.88,30.9,-1.74,75.0,8.15,10.14,1.66,4.20,1.206000e+07,28115262.0,True,False,True,True,True,True,False,True,True,True,True,True,10
1916,SNFCA,8.035180,8.530577,9.405556,40.11,13.15,682.6,2.71,93.9,10.35,10.54,3.76,44.40,1.887000e+07,218595.0,False,True,True,True,True,False,True,True,True,True,True,True,10
